In [1]:
import os
os.chdir('../')
%pwd

'/Users/isma/Documents/Portfolio/kitchen_robot'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModelConfig:
    root_dir: Path
    feature_size: int
    target_size: int
    base_model_path: Path

In [10]:
from kitchen_robot.constants import *
# from kitchen_robot.entity.config_entity import (
#     PrepareDatasetConfig,
# )
from kitchen_robot.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        print(config_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_model_config(self) -> PrepareModelConfig:
        config = self.config.prepare_model
        create_directories([config.root_dir])

        prepare_model_config = PrepareModelConfig(
            root_dir=config.root_dir,
            feature_size=config.feature_size,
            target_size=config.target_size,
            base_model_path=config.base_model_path,
        )
        print(prepare_model_config)

        return prepare_model_config

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

class PredictionModel(nn.Module):
    def __init__(self, feature_size : int, target_size : int):
        super(PredictionModel, self).__init__()
        self.fc1 = nn.Linear(feature_size, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, target_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [15]:
class PrepareModel:
    def __init__(self, config: PrepareModelConfig):
        self.config = config
        print(config)
    
    def generate_base_model(self):
        self.model = PredictionModel(self.config.feature_size, self.config.target_size)
        torch.save(self.model.state_dict(), self.config.base_model_path)

In [16]:
config = ConfigurationManager()
prepare_model_config = config.get_prepare_model_config()
print(prepare_model_config)
prepare_model = PrepareModel(prepare_model_config)
prepare_model.generate_base_model()

config/config.yaml
[2024-02-27 23:08:41,088: INFO: common: created directory at: artifacts]
[2024-02-27 23:08:41,089: INFO: common: created directory at: artifacts/prepare_model]
PrepareModelConfig(root_dir='artifacts/prepare_model', feature_size=5, target_size=2, base_model_path='artifacts/prepare_model/base_model.pth')
PrepareModelConfig(root_dir='artifacts/prepare_model', feature_size=5, target_size=2, base_model_path='artifacts/prepare_model/base_model.pth')
PrepareModelConfig(root_dir='artifacts/prepare_model', feature_size=5, target_size=2, base_model_path='artifacts/prepare_model/base_model.pth')
